In [2]:
!python --version

Python 3.11.10


In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
#import random
import numpy as np
import pandas as pd

In [5]:
from line_profiler import LineProfiler

In [6]:
import matplotlib.pyplot as plt

In [7]:
%matplotlib inline

In [8]:
class World():
    """Class World - Макромир, который задает начало отсчета времени,
        законы макроэкономики, ограничения регуляторов/ЦБ и остальное окружение.
        Так же здесь описывается поведение клиента, которое может зависеть от внешних факторов.

        dod_migration - матрица вероятностей переходов по просрочкам
    """

    def __init__(self):
        self.World_Time = 0
        #                             0    1+   31+   61+   91+   WOF
        self.dod_migration = np.array([[0.95, 0.05, 0.00, 0.00, 0.00, 0.00], #  0 
                                       [0.90, 0.05, 0.05, 0.00, 0.00, 0.00], #  1+
                                       [0.10, 0.05, 0.05, 0.80, 0.00, 0.00], # 31+
                                       [0.05, 0.05, 0.05, 0.05, 0.80, 0.00], # 61+
                                       [0.01, 0.01, 0.02, 0.02, 0.04, 0.90], # 91+
                                       [0.00, 0.00, 0.00, 0.00, 0.00, 1.00]  # WOF
            #                          [0.00, 0.00, 0.00, 0.00, 0.00, 1.00]  # TODO - добавить досрочное и часичнодочсрочное погашение кредита 
                         ])
        assert self.dod_migration.shape[0] == self.dod_migration.shape[1] # проверка на квадратность
        assert [i.sum() for i in self.dod_migration] == [1 for i in range(len(self.dod_migration))] # 1 in sum of row
        print('test - ', [i.sum() for i in self.dod_migration])

In [9]:
class DWH_DB():
    """Class DWH - база данных
    """
    def __init__(self):
        self.LI = pd.DataFrame(columns = ['CNTR_ID',
                                          'SD',
                                          'DOD_ID',
                                          'MOB',
                                          'WRTOFF_ID',
                                          'CLOSED_ID'
                                          ])
        self.DMContract = pd.DataFrame(columns = ['CNTR_ID',
                                                  'ISSUE_DT',
                                                  'WRTOFF_DT',
                                                  'CLOSED_DT'
                                                 ])
        


In [10]:
class Contract():
    """Class Contract
       issue_dt - issue of contract
       duration - duration in months
    """

    dod_dic = {0: '0',
               1: '1+',
               2: '31+',
               3: '61+',
               4: '91+',
               5: 'WOF'
              }
    dod_cnt = 6 # кол-во состояний
    dod_states = np.eye(dod_cnt) # матрица состояний (для удобства использована единичная матрица)

    def __init__(self, cntr_id = 0, issue_dt = 0, duration = 0,
                 world = World):
        self.cntr_id = cntr_id
        self.dod_id = 0        # начальное состояние контракта при выдачи: DOD = 0
        self.dod_state = self.dod_states[0] # np.array([1,0,0,0,0]) 
        self.dod_migration = world.dod_migration
        self.issue_dt = issue_dt
        self.mob = 0
        self.duration = duration
        self.closed_id = 0       # 0 - контратк открыт, 1 - закрыт
        self.wrtoff_id = 0       # 0 - контратк несписан, 1 - списан
        
    def next_month(self):
        if self.closed_id == 1:
            return None
           
        self.mob = self.mob + 1
        p = self.dod_migration.T.dot(self.dod_state) # array of probabilities
        self.dod_id = np.random.choice(self.dod_cnt,1,p=p)[0] # new state
        self.dod_state = self.dod_states[self.dod_id]

        if self.dod_id == 0 and self.mob >= self.duration: # погашение либо выздоровление с возвращением в график
            self.closed_id = 1
        
        if self.dod_id == 5 and self.mob >= self.duration + 12: # списание
            self.wrtoff_id = 1

        if self.wrtoff_id == 1 and self.mob >= self.duration + 24: # закрытие списанного контракта
            self.closed_id = 1
            

In [11]:
class Portfolio():
    """Class Portfolio - Портфель - динамика 
        N - первая выдача при создании портфеля
        start_portfolio_dt - привязка портфеля к мировому времени - важно при наличии нескольких портфелей
    
    """
    def __init__(self, N = 10, duration = 36, start_portfolio_dt = 0, world = None):
        self.cntr_id = 0                                # счетчик контрактов
        self.start_portfolio_dt = start_portfolio_dt    # дата создания портфеля
        self.cntr_list = []                             # сам портфель - список контрактов
        self.portfolio_age = 0                          # возрвст портфеля
        self.world = world

        # проведем первую выдачу - инициализация портфеля
        self.issue(N, duration)
        # Заполним LI
        self.fix_in_dwh()

    def issue(self, N = 10, duration = 36):
        for i in range(N):
            self.cntr_id += 1
            self.cntr_list.append(Contract(cntr_id = self.cntr_id, 
                                           issue_dt = self.start_portfolio_dt,
                                           duration = duration,
                                           world = self.world))

    def next_month(self, N = 10, duration = 36, log = False):
        self.portfolio_age +=1

        # Для проверки - выведем все закрытые на этот момент контракты
        if log:
            test = [cntr.cntr_id for cntr in self.cntr_list if cntr.closed_id == 1 ]
            print('%04i' % self.portfolio_age, len(self.cntr_list), 'out ->',  test)            

        # Перезапишем список только открытыми контрактами 
        self.cntr_list = [cntr for cntr in self.cntr_list if cntr.closed_id == 0 ]
        
        # сдвинем существующий портфель, потом проведем выдачу новых         
        for cntr in self.cntr_list:
            cntr.next_month()
            
        # проведем выдачи
        self.issue(N, duration)

        # Заполним LI
        self.fix_in_dwh()


    def fix_in_dwh_old(self): # Пример медленной вставки
        ix = len(DWH.LI.index)
        for cnt in self.cntr_list:
            DWH.LI.loc[ix] = [cnt.cntr_id, self.portfolio_age, cnt.dod_id, cnt.mob]
            ix += 1

    def fix_in_dwh(self):
        fix_data = [[cnt.cntr_id, self.portfolio_age, cnt.dod_id, cnt.mob, cnt.wrtoff_id, cnt.closed_id] for cnt in self.cntr_list]
        DWH.LI = pd.concat([DWH.LI,
                            pd.DataFrame(data=fix_data,
                                         columns=DWH.LI.columns)
                           ])


In [12]:
%time
N_const = 1000 # Пусть будут постоянные ежемесячные выдачи 
duration = 24
WW = World()
DWH = DWH_DB()
GP = Portfolio(N_const, duration, world = WW)

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 10 μs
test -  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [13]:
for t in range(100):
    GP.next_month(N_const, duration)

In [14]:
T = DWH.LI.reset_index(drop=True)
print(T.shape)
T.tail()

(2296191, 6)


,CNTR_ID,SD,DOD_ID,MOB,WRTOFF_ID,CLOSED_ID
2296186,100996,100,0,0,0,0
2296187,100997,100,0,0,0,0
2296188,100998,100,0,0,0,0
2296189,100999,100,0,0,0,0
2296190,101000,100,0,0,0,0


In [15]:
T[T['CNTR_ID']==7]

,CNTR_ID,SD,DOD_ID,MOB,WRTOFF_ID,CLOSED_ID
6,7,0,0,0,0,0
1006,7,1,0,1,0,0
3006,7,2,0,2,0,0
6006,7,3,0,3,0,0
10006,7,4,0,4,0,0
15006,7,5,0,5,0,0
21006,7,6,0,6,0,0
28006,7,7,0,7,0,0
36006,7,8,1,8,0,0
45006,7,9,0,9,0,0


In [16]:
G = T.groupby('SD')[['MOB']].agg(['count','mean']).reset_index()
G.columns = ['SD','CNT','MEAN']
G

,SD,CNT,MEAN
0,0,1000,0.0
1,1,2000,0.5
2,2,3000,1.0
3,3,4000,1.5
4,4,5000,2.0
...,...,...,...
96,96,26076,12.989224
97,97,26076,12.983318
98,98,26031,12.948177
99,99,26043,12.956687


In [17]:
G.groupby('CNT')['SD'].agg(['count','min']).tail(20)

,count,min
CNT,,
26105,1,68
26106,1,80
26108,2,56
26110,1,88
26112,1,57
26113,2,54
26115,1,63
26116,1,78
26117,1,66


In [18]:
m_lag = 1  # период переката

ix_sd   = T['SD'] >= 40 # для кредитов сроком 24 мес
#ix_dod  = T['DOD_ID'] < 4 # не дефолты
ix_open = T['CLOSED_ID'] == 0 # не закрытые
ix_mlag = T['SD'] <= (T['SD'].max()-m_lag) # ограничение на вызревание
D_F = T.loc[ix_sd & ix_mlag & ix_open, :]

In [19]:
D_F

,CNTR_ID,SD,DOD_ID,MOB,WRTOFF_ID,CLOSED_ID
705860,15,40,5,40,1,0
705861,77,40,5,40,1,0
705862,85,40,5,40,1,0
705863,108,40,5,40,1,0
705864,120,40,5,40,1,0
...,...,...,...,...,...,...
2270127,99996,99,0,0,0,0
2270128,99997,99,0,0,0,0
2270129,99998,99,0,0,0,0
2270130,99999,99,0,0,0,0


In [20]:
D_N = T.copy()
D_N['SD_OLD'] = D_N['SD']
D_N['SD'] = D_N['SD']-1

In [21]:
D_N

,CNTR_ID,SD,DOD_ID,MOB,WRTOFF_ID,CLOSED_ID,SD_OLD
0,1,-1,0,0,0,0,0
1,2,-1,0,0,0,0,0
2,3,-1,0,0,0,0,0
3,4,-1,0,0,0,0,0
4,5,-1,0,0,0,0,0
...,...,...,...,...,...,...,...
2296186,100996,99,0,0,0,0,100
2296187,100997,99,0,0,0,0,100
2296188,100998,99,0,0,0,0,100
2296189,100999,99,0,0,0,0,100


In [22]:
D = D_F.merge(D_N, on = ['CNTR_ID','SD'], how = 'left', suffixes=['_F','_N'])

In [23]:
D[D['DOD_ID_N'].isna()]

,CNTR_ID,SD,DOD_ID_F,MOB_F,WRTOFF_ID_F,CLOSED_ID_F,DOD_ID_N,MOB_N,WRTOFF_ID_N,CLOSED_ID_N,SD_OLD


In [24]:
D['DOD_F'] = D['DOD_ID_F'].apply(lambda x: 'D_%s'%x)
D['DOD_N'] = D['DOD_ID_N'].apply(lambda x: 'D_%s'%x)

In [25]:
M = D.groupby([#'SD',
               'DOD_F','WRTOFF_ID_F','CLOSED_ID_F','DOD_N','WRTOFF_ID_N','CLOSED_ID_N']
             )['CNTR_ID'].count().reset_index()
M

,DOD_F,WRTOFF_ID_F,CLOSED_ID_F,DOD_N,WRTOFF_ID_N,CLOSED_ID_N,CNTR_ID
0,D_0,0,0,D_0,0,0,1225826
1,D_0,0,0,D_0,0,1,51848
2,D_0,0,0,D_1,0,0,66622
3,D_1,0,0,D_0,0,0,58029
4,D_1,0,0,D_0,0,1,5431
5,D_1,0,0,D_1,0,0,3467
6,D_1,0,0,D_2,0,0,3537
7,D_2,0,0,D_0,0,0,320
8,D_2,0,0,D_0,0,1,68
9,D_2,0,0,D_1,0,0,210


In [26]:
MM = M.pivot_table(index='DOD_F', columns='DOD_N', values='CNTR_ID', aggfunc='sum', fill_value=0)
MM

DOD_N,D_0,D_1,D_2,D_3,D_4,D_5
DOD_F,,,,,,
D_0,1277674,66622,0,0,0,0
D_1,63460,3467,3537,0,0,0
D_2,388,210,203,3129,0,0
D_3,169,148,166,180,2704,0
D_4,23,29,59,56,103,2548
D_5,0,0,0,0,0,79745


In [27]:
MM['SUM'] = MM.sum(axis=1)

In [28]:
MM

DOD_N,D_0,D_1,D_2,D_3,D_4,D_5,SUM
DOD_F,,,,,,,
D_0,1277674,66622,0,0,0,0,1344296
D_1,63460,3467,3537,0,0,0,70464
D_2,388,210,203,3129,0,0,3930
D_3,169,148,166,180,2704,0,3367
D_4,23,29,59,56,103,2548,2818
D_5,0,0,0,0,0,79745,79745


In [114]:
for c in MM.columns:
    MM[c] = MM[c]/MM['SUM']
MM.drop(columns='SUM', inplace=True)    
MM

DOD_N,D_0,D_1,D_2,D_3,D_4,D_5
DOD_F,,,,,,
D_0,0.950160,0.049840,0.000000,0.000000,0.000000,0.000000
D_1,0.899629,0.049280,0.051091,0.000000,0.000000,0.000000
D_2,0.102690,0.049450,0.049820,0.798040,0.000000,0.000000
D_3,0.047903,0.050937,0.051696,0.050396,0.799068,0.000000
D_4,0.010025,0.008983,0.022914,0.018617,0.039839,0.899622
D_5,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
